In [2]:
import pandas as pd

## Carregamento das bases de dados utilizadas

### Exportação

In [3]:
exportacao_geral = pd.read_excel('./dados/comex/Exportacao Mundo 2024 NCM.xlsx')
exportacao_geral['Código NCM'] = exportacao_geral['Código NCM'].astype(str)
exportacao_geral['Código NCM'] = exportacao_geral['Código NCM'].str.zfill(8)

exportacao_usa = pd.read_excel('./dados/comex/Exportacao EUA 2024 NCM.xlsx')
exportacao_usa['Código NCM'] = exportacao_usa['Código NCM'].astype(str)
exportacao_usa['Código NCM'] = exportacao_usa['Código NCM'].str.zfill(8)

In [6]:
exportacao_usa.head(2)

,Código NCM,Descrição NCM,2024 - Valor US$ FOB
0,27090010,Óleos brutos de petróleo,5830983860
1,72071200,Outros produtos semimanufaturados de ferro ou ...,2774470802


In [7]:
exportacao_usa.dtypes

Código NCM               int64
Descrição NCM           object
2024 - Valor US$ FOB     int64
dtype: object

## Prodlist

In [4]:
mapeamento_prodlist = pd.read_excel('./dados/mapeamento/PRODLIST Ind 2022 x NCM 2022.xlsx')

In [5]:
ncms_exp_usa = exportacao_usa['Código NCM'].unique().tolist()

Verificar NCM's que pertencem a mais de uma Prodlist e se isso representa um problema para o mapeamento para o SCN ( Mais de um produto ou serviço)

In [6]:
mapeamento_prodlist['prodlist 6d'] = mapeamento_prodlist['PRODLIST-Ind 2022'].str[0:7]
mapeamento_prodlist['prodlist 4d'] = mapeamento_prodlist['PRODLIST-Ind 2022'].str[0:4]
mapeamento_prodlist['prodlist 2d'] = mapeamento_prodlist['PRODLIST-Ind 2022'].str[0:2]


In [7]:
mapeamento_prodlist['NCM'] = mapeamento_prodlist['NCM 2022 (SET/2022)'].str.replace('.', '')


In [8]:
mapeamento_prodlist = mapeamento_prodlist.loc[mapeamento_prodlist['NCM'].isin(ncms_exp_usa), :]

In [9]:

analise_mapeamento = mapeamento_prodlist.groupby('NCM').nunique()['prodlist 4d'].reset_index()

lista_ncms_mais_1_prodlist = analise_mapeamento.loc[analise_mapeamento['prodlist 4d'] > 1, :].sort_values('prodlist 4d', ascending=False)['NCM'].to_list()


mapeamento_varias_prodlist = mapeamento_prodlist.loc[mapeamento_prodlist['NCM'].isin(lista_ncms_mais_1_prodlist), :]



In [10]:
analise_mapeamento.sort_values('prodlist 4d', ascending=False).head(10)

,NCM,prodlist 4d
2909,76169900,6
4273,87089990,6
2864,75089090,5
2921,80070090,5
2916,79070090,5
249,16029000,4
240,16010000,4
295,19041000,4
296,19042000,4
2913,78060090,4


In [11]:
mapeamento_varias_prodlist.query('NCM=="76169900"')

,PRODLIST-Ind 2022,NCM 2022 (SET/2022),prodlist 6d,prodlist 4d,prodlist 2d,NCM
8770,2452.2010,7616.99.00,2452.20,2452,24,76169900
8789,2512.2020,7616.99.00,2512.20,2512,25,76169900
8825,2531.2020,7616.99.00,2531.20,2531,25,76169900
8835,2532.2030,7616.99.00,2532.20,2532,25,76169900
9012,2592.2030,7616.99.00,2592.20,2592,25,76169900
9094,2599.2020,7616.99.00,2599.20,2599,25,76169900
9150,2599.2090,7616.99.00,2599.20,2599,25,76169900


In [12]:
mapeamento_prodlist['NCM'] = mapeamento_prodlist['NCM 2022 (SET/2022)'].str.replace('.', '', regex=False)

In [13]:
mapeamento_prodlist.shape

(6183, 6)

In [14]:
mapeamento_prodlist.head()

,PRODLIST-Ind 2022,NCM 2022 (SET/2022),prodlist 6d,prodlist 4d,prodlist 2d,NCM
5,0600.2010,2711.11.00,0600.20,0600,06,27111100
8,0600.2030,2709.00.10,0600.20,0600,06,27090010
9,0600.2040,2714.10.00,0600.20,0600,06,27141000
10,0710.2015,2601.11.00,0710.20,0710,07,26011100
11,0710.2015,2601.12.10,0710.20,0710,07,26011210


In [15]:
exportacao_usa.head()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB
0,27090010,Óleos brutos de petróleo,5830983860
1,72071200,Outros produtos semimanufaturados de ferro ou ...,2774470802
2,09011110,"Café não torrado, não descafeinado, em grão",1895591133
3,47032900,"Pastas químicas de madeira, à soda ou ao sulfa...",1551604956
4,72011000,"Ferro fundido bruto não ligado, que contenha, ...",1423332945


In [16]:
df_participacao = exportacao_geral.copy().merge(exportacao_usa.copy().drop(['Descrição NCM'],axis=1).rename(columns={'2024 - Valor US$ FOB': 'Valor FOB USA'}), on='Código NCM')



In [17]:
df_participacao['Participação USA'] = df_participacao['Valor FOB USA'] / df_participacao['2024 - Valor US$ FOB']

In [18]:
df_participacao.sort_values(by='Participação USA', ascending=False, inplace=True)

In [19]:
lista_empresas = pd.read_excel('./dados/comex/Lista de empresas.xlsx')

## Preparar a junção de dados

In [40]:
df_participacao.tail()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA
5021,29189911,"Ácido fenoxiacético, seus sais e seus ésteres",52,0,0.0
5031,13021400,"Sucos e extratos, de piretro ou de raízes com ...",3,0,0.0
5032,29161511,Oleato de manitol,0,0,NaN
5033,29329992,"1,3,4,6,7,8-Hexaidro-4,6,6,7,8,8-hexametilcicl...",0,0,NaN
5034,33012913,"Óleo essencial, de pau-santo (Bulnesia sarmien...",0,0,NaN


In [21]:
df_participacao.shape

(5035, 5)

In [22]:
print()
df_participacao.merge(mapeamento_prodlist[['NCM','prodlist 4d']].drop_duplicates().rename(columns={'NCM' : 'Código NCM', 'prodlist 4d' : 'cnae'}), on='Código NCM',indicator=True,how='left')[['_merge']].value_counts('_merge')

_merge
both          5437
left_only      251
right_only       0
Name: count, dtype: int64

In [23]:
df_participacao.merge(mapeamento_prodlist[['NCM','prodlist 4d']].drop_duplicates().rename(columns={'NCM' : 'Código NCM', 'prodlist 4d' : 'cnae'}), on='Código NCM',indicator=True,how='left').head()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA,cnae,_merge
0,48111010,"Papel e cartão de celulose, alcatroados, betum...",11,11,1.0,1721,both
1,48111010,"Papel e cartão de celulose, alcatroados, betum...",11,11,1.0,1722,both
2,50072090,"Outros tecidos que contenham pelo menos 85 %, ...",151,151,1.0,1314,both
3,50072090,"Outros tecidos que contenham pelo menos 85 %, ...",151,151,1.0,1322,both
4,29221700,Metildietanolamina e etildietanolamina,10,10,1.0,2029,both


In [24]:
df_participacao.merge(mapeamento_prodlist[['NCM','prodlist 4d']].drop_duplicates().rename(columns={'NCM' : 'Código NCM', 'prodlist 4d' : 'cnae'}), on='Código NCM',indicator=True,how='left').tail()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA,cnae,_merge
5683,29189911,"Ácido fenoxiacético, seus sais e seus ésteres",52,0,0.0,2029,both
5684,13021400,"Sucos e extratos, de piretro ou de raízes com ...",3,0,0.0,1099,both
5685,29161511,Oleato de manitol,0,0,NaN,2029,both
5686,29329992,"1,3,4,6,7,8-Hexaidro-4,6,6,7,8,8-hexametilcicl...",0,0,NaN,2029,both
5687,33012913,"Óleo essencial, de pau-santo (Bulnesia sarmien...",0,0,NaN,2093,both


In [25]:
df_participacao_empresas = df_participacao.merge(mapeamento_prodlist[['NCM','prodlist 4d']].drop_duplicates().rename(columns={'NCM' : 'Código NCM', 'prodlist 4d' : 'cnae'}), on='Código NCM',how='left')

In [26]:
df_participacao_empresas.head(2)

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA,cnae
0,48111010,"Papel e cartão de celulose, alcatroados, betum...",11,11,1.0,1721
1,48111010,"Papel e cartão de celulose, alcatroados, betum...",11,11,1.0,1722


In [28]:
lista_empresas['co_cnae'] = lista_empresas['co_cnae'].astype(str)
lista_empresas['co_cnae'] = lista_empresas['co_cnae'].str.zfill(4)

In [29]:
lista_empresas.head(2)

,co_oper,no_razao_soc,co_cnae,no_cnae
0,43201151001001,CUMMINS BRASIL LIMITADA,4530,Comércio de peças e acessórios para veículos a...
1,3622151000170,NEVESCO IMPORTACAO EXPORTACAO LTDA,4651,"Comércio atacadista de computadores, periféric..."


In [65]:
lista_empresas.merge(df_participacao_empresas,right_on='cnae',left_on='co_cnae',how='left').to_excel('./dados/lista_empresas_mapeamento_exportacao.xlsx')

In [42]:
lista_empresa_participacao = lista_empresas.merge(df_participacao_empresas,right_on='cnae',left_on='co_cnae',how='outer',indicator=True)



In [47]:
lista_empresa_participacao.query('_merge == "left_only"')['co_cnae'].drop_duplicates()

0         0111
2         0115
3         0119
4         0121
5         0131
          ... 
196445    9602
196449    9609
196450    9900
196453    9904
196455    9999
Name: co_cnae, Length: 302, dtype: object

In [ ]:
lista_empresa_participacao.query('_merge == "right_only"')['cnae'].drop_duplicates()

(14,)

In [41]:
lista_empresa_participacao.query('_merge == "left_only"')[['co_cnae','no_cnae']].drop_duplicates()

,co_cnae,no_cnae
0,4530,Comércio de peças e acessórios para veículos a...
1,4651,"Comércio atacadista de computadores, periféric..."
312,7732,Aluguel de máquinas e equipamentos para constr...
313,4645,Comércio atacadista de instrumentos e materiai...
314,5112,Transporte aéreo de passageiros nãoregular
...,...,...
186368,8291,Atividades de cobrança e informações cadastrais
186573,5021,Transporte por navegação interior de carga
190755,7911,Agências de viagens
191229,1340,"Acabamentos em fios, tecidos e artefatos têxteis"


In [68]:
df_participacao_empresas.query('`Código NCM`=="09012200"')

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA,cnae
1460,09012200,"Café torrado, descafeinado",201344,66459,0.330077,1081
1461,09012200,"Café torrado, descafeinado",201344,66459,0.330077,1082


In [74]:
participacao_cnae = pd.read_excel('./dados/participacao_exportacao_americana_pia_empresa_cnae.xlsx',dtype={'cnae': str})

In [89]:
participacao_cnae.head(2)

,cnae,cnae_descricao,Receita líquida de vendas de produtos e serviços industriais,Total da receita líquida de vendas,2023 - Valor R$ FOB Mundo,2023 - Valor R$ FOB USA
0,050,Extração de carvão mineral,1850253000,1941673000,1.776615e+06,0.000000e+00
1,060,Extração de petróleo e gás natural,162636312000,163281926000,2.128577e+11,2.337750e+10


In [88]:
lista_empresa_participacao.head(2)

,co_oper,no_razao_soc,co_cnae,no_cnae,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA,cnae
0,43201151001001,CUMMINS BRASIL LIMITADA,4530,Comércio de peças e acessórios para veículos a...,NaN,NaN,NaN,NaN,NaN,NaN
1,3622151000170,NEVESCO IMPORTACAO EXPORTACAO LTDA,4651,"Comércio atacadista de computadores, periféric...",NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# fazer merge com o casamento parcial de chaves.
lista_empresa_participacao_setor = lista_empresa_participacao.drop('cnae',axis=1).merge(participacao_cnae, left_on=lista_empresa_participacao['co_cnae'].str[0:3],right_on='cnae', how='inner')

In [94]:
lista_empresa_participacao.head(2)

,co_oper,no_razao_soc,co_cnae,no_cnae,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA,cnae
0,43201151001001,CUMMINS BRASIL LIMITADA,4530,Comércio de peças e acessórios para veículos a...,NaN,NaN,NaN,NaN,NaN,NaN
1,3622151000170,NEVESCO IMPORTACAO EXPORTACAO LTDA,4651,"Comércio atacadista de computadores, periféric...",NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
lista_empresa_participacao['no_razao_soc'].drop_duplicates().shape

(9119,)

In [96]:

lista_empresa_participacao_setor['no_razao_soc'].drop_duplicates().shape

(5379,)

In [99]:
lista_empresa_participacao_setor['Participacao Exp USAx RLV'] = lista_empresa_participacao_setor['2023 - Valor R$ FOB USA'] / lista_empresa_participacao_setor['Total da receita líquida de vendas']

In [108]:
lista_empresa_participacao_setor.head(2)

,co_oper,no_razao_soc,co_cnae,no_cnae,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA,cnae,cnae_descricao,Receita líquida de vendas de produtos e serviços industriais,Total da receita líquida de vendas,2023 - Valor R$ FOB Mundo,2023 - Valor R$ FOB USA,Participacao Exp USAx RLV
0,20003699000150,FS AGRISOLUTIONS INDUSTRIA DE BIOCOMBUSTIVEIS ...,1931,Fabricação de álcool,22071010,"Álcool etílico não desnaturado, com um teor al...",331809827.0,160843628.0,0.484746,193,Fabricação de biocombustíveis,91664322000,105389246000,9.095401e+09,1.657803e+09,0.01573
1,20003699000150,FS AGRISOLUTIONS INDUSTRIA DE BIOCOMBUSTIVEIS ...,1931,Fabricação de álcool,22089000,Outras bebidas alcoólicas,36770823.0,6256550.0,0.170150,193,Fabricação de biocombustíveis,91664322000,105389246000,9.095401e+09,1.657803e+09,0.01573


In [102]:
lista_empresa_participacao_setor.query('`Participação USA`.isnull()').shape

(363, 16)

In [111]:
lista_empresa_participacao_setor.head(2)

,co_oper,no_razao_soc,co_cnae,no_cnae,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA,cnae,cnae_descricao,Receita líquida de vendas de produtos e serviços industriais,Total da receita líquida de vendas,2023 - Valor R$ FOB Mundo,2023 - Valor R$ FOB USA,Participacao Exp USAx RLV
0,20003699000150,FS AGRISOLUTIONS INDUSTRIA DE BIOCOMBUSTIVEIS ...,1931,Fabricação de álcool,22071010,"Álcool etílico não desnaturado, com um teor al...",331809827.0,160843628.0,0.484746,193,Fabricação de biocombustíveis,91664322000,105389246000,9.095401e+09,1.657803e+09,0.01573
1,20003699000150,FS AGRISOLUTIONS INDUSTRIA DE BIOCOMBUSTIVEIS ...,1931,Fabricação de álcool,22089000,Outras bebidas alcoólicas,36770823.0,6256550.0,0.170150,193,Fabricação de biocombustíveis,91664322000,105389246000,9.095401e+09,1.657803e+09,0.01573


In [112]:
##Criar um arquivo excel com duas abas, uma que atenda ao critério abaixo e outra em que o código de NCM é nulo.
lista_empresa_participacao_setor.query('`Participação USA` > 0.5 and `Participacao Exp USAx RLV` > .15')['no_razao_soc'].drop_duplicates().shape    
lista_empresa_participacao_setor.query('`Participação USA` > 0.5 and `Participacao Exp USAx RLV` > .15')['no_razao_soc'].drop_duplicates().shape

with pd.ExcelWriter('./dados/Lista Empresas- Final.xlsx') as writer:
    lista_empresa_participacao_setor.query('`Participação USA` > 0.5 and `Participacao Exp USAx RLV` > .15')[['co_oper','no_razao_soc','co_cnae','no_cnae']].to_excel(writer, sheet_name='Critérios aplicados', index=False)
    lista_empresa_participacao_setor[lista_empresa_participacao_setor['Código NCM'].isnull()][['co_oper','no_razao_soc','co_cnae','no_cnae']].to_excel(writer, sheet_name='Sem correlação', index=False)  



In [98]:
lista_empresa_participacao_setor[lista_empresa_participacao_setor['Código NCM'].isnull()]

,co_oper,no_razao_soc,co_cnae,no_cnae,Código NCM,Descrição NCM,2024 - Valor US$ FOB,Valor FOB USA,Participação USA,cnae,cnae_descricao,Receita líquida de vendas de produtos e serviços industriais,Total da receita líquida de vendas,2023 - Valor R$ FOB Mundo,2023 - Valor R$ FOB USA
690,28903129000129,COVINGTON AIRCRAFT DO BRASIL LTDA,3316,Manutenção e reparação de aeronaves,NaN,NaN,NaN,NaN,NaN,331,Manutenção e reparação de máquinas e equipamentos,50921841000,54657916000,0.0,0.0
767,48090120000153,SAFRAN HELICOPTER ENGINES INDUSTRIA E COMERCIO...,3316,Manutenção e reparação de aeronaves,NaN,NaN,NaN,NaN,NaN,331,Manutenção e reparação de máquinas e equipamentos,50921841000,54657916000,0.0,0.0
1781,33435231000187,GE CELMA LTDA.,3316,Manutenção e reparação de aeronaves,NaN,NaN,NaN,NaN,NaN,331,Manutenção e reparação de máquinas e equipamentos,50921841000,54657916000,0.0,0.0
2082,6270238000113,GC - SERVICOS DE AUTOMACAO EIRELI,3321,Instalação de máquinas e equipamentos industriais,NaN,NaN,NaN,NaN,NaN,332,Instalação de máquinas e equipamentos,6735735000,7197921000,0.0,0.0
2316,5907602000140,SGA SOLUCOES EM ENERGIA E COMERCIO LTDA,3321,Instalação de máquinas e equipamentos industriais,NaN,NaN,NaN,NaN,NaN,332,Instalação de máquinas e equipamentos,6735735000,7197921000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187197,3013973000315,UNIFI DO BRASIL LTDA.,1340,"Acabamentos em fios, tecidos e artefatos têxteis",NaN,NaN,NaN,NaN,NaN,134,"Acabamentos em fios, tecidos e artefatos têxteis",4918213000,4985349000,0.0,0.0
189019,22093184000114,S3 DO BRASIL - SERVICOS DE MANUTENCAO AERONAUT...,3312,Manutenção e reparação de equipamentos eletrôn...,NaN,NaN,NaN,NaN,NaN,331,Manutenção e reparação de máquinas e equipamentos,50921841000,54657916000,0.0,0.0
189178,36243599000131,AXPR VALVE SCIENCE DISTRIBUICAO E MANUTENCAO I...,3314,Manutenção e reparação de máquinas e equipamen...,NaN,NaN,NaN,NaN,NaN,331,Manutenção e reparação de máquinas e equipamentos,50921841000,54657916000,0.0,0.0
189179,18101019000190,GEORG ANDREAS JOOS EGAS 01219953997 ...,3329,Instalação de equipamentos não especificados a...,NaN,NaN,NaN,NaN,NaN,332,Instalação de máquinas e equipamentos,6735735000,7197921000,0.0,0.0
